<a href="https://colab.research.google.com/github/alhichri/RS-DeepSuperLearner/blob/main/convert_large_datasets_to_tfrecords.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import IPython.display as display
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import os
import pathlib
import glob

from sklearn.metrics import confusion_matrix,  classification_report, accuracy_score
from sklearn.utils.multiclass import unique_labels
from sklearn.utils import shuffle

tf.__version__


from google.colab import drive
drive.mount('/content/gdrive' , force_remount=True)

Mounted at /content/gdrive


In [ ]:
# import pathlib
# data_dir = tf.keras.utils.get_file(origin='https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz',
#                                          fname='flower_photos', untar=True)
# print(data_dir)

# data_dir = pathlib.Path(data_dir)
# print(data_dir)

datatsets_path = "/content/gdrive/My Drive/RS_datasets/"
# dataset = 'AID'; image_format = '.png';   folds_split_text_file_name = 'AID_folds_split_text_file.txt'
# dataset = 'PatternNet' ; image_format = '.jpg';     folds_split_text_file_name = 'PatternNet_folds_split_text_file.txt'
dataset = 'NWPU_RESISC45' ; image_format = '.jpg';      folds_split_text_file_name = 'NWPU_RESISC45_folds_split_text_file.txt'

data_dir = datatsets_path+dataset+ '/'; print(data_dir)
data_dir = pathlib.Path(data_dir)

image_count = len(list(data_dir.glob('*/*'+image_format)))

CLASS_NAMES = np.array([item.name for item in data_dir.glob('*') if item.name != "LICENSE.txt"])
num_of_classes = len(CLASS_NAMES)
print(CLASS_NAMES)
print(num_of_classes)

class_size = np.zeros((num_of_classes)).astype('int')
for cn in range(num_of_classes):
    path_name = datatsets_path+dataset + '/' + CLASS_NAMES[cn]
    data_dir = pathlib.Path(path_name)
    class_size[cn] = len(list(data_dir.glob('*'+image_format)))
    print('path name and class size: ' , path_name , class_size[cn])


cn = 2
path_name = datatsets_path+dataset + '/' + CLASS_NAMES[cn]
data_dir = pathlib.Path(path_name)
file_names_with_paths = list(data_dir.glob('*'+image_format))
# for i in range(class_size[cn]):
#     print(file_names_with_paths[i])






In [ ]:
print('num of classes: ', num_of_classes)
print('Class names: ', CLASS_NAMES)
print('Total number of images: ' , image_count)
for cn in range(num_of_classes):
    print('Class name and size: ' , CLASS_NAMES[cn] , class_size[cn])

num of classes:  45
Class names:  ['lake' 'palace' 'chaparral' 'dense_residential' 'parking_lot'
 'tennis_court' 'island' 'desert' 'meadow' 'beach' 'thermal_power_station'
 'cloud' 'terrace' 'sea_ice' 'snowberg' 'roundabout' 'sparse_residential'
 'runway' 'airplane' 'storage_tank' 'overpass' 'church' 'wetland' 'bridge'
 'commercial_area' 'intersection' 'basketball_court' 'freeway' 'mountain'
 'railway' 'airport' 'circular_farmland' 'baseball_diamond'
 'mobile_home_park' 'rectangular_farmland' 'railway_station' 'river'
 'ground_track_field' 'industrial_area' 'stadium' 'ship' 'harbor'
 'medium_residential' 'forest' 'golf_course']
Total number of images:  31503
Class name and size:  lake 700
Class name and size:  palace 700
Class name and size:  chaparral 700
Class name and size:  dense_residential 700
Class name and size:  parking_lot 700
Class name and size:  tennis_court 700
Class name and size:  island 700
Class name and size:  desert 700
Class name and size:  meadow 700
Class name an

In [ ]:
#################################################################################################
####### This code reads images and saves them in tfrecords files
####### Data is  divided into folds and each fold is saved in a tfrecords
#################################################################################################
nfolds = 20;

def _bytes_feature(value):
  """Returns a bytes_list from a string / byte."""
  if isinstance(value, type(tf.constant(0))):
    value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
  """Returns a float_list from a float / double."""
  return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _int64_feature(value):
  """Returns an int64_list from a bool / enum / int / uint."""
  return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

# Create a dictionary with features that may be relevant.
def image_example(image_string, label):
  image_shape = tf.image.decode_jpeg(image_string).shape

  feature = {
      #'height': _int64_feature(image_shape[0]),
      #'width': _int64_feature(image_shape[1]),
      #'depth': _int64_feature(image_shape[2]),
      'label': _int64_feature(label),
      'image_raw': _bytes_feature(image_string),
  }

  return tf.train.Example(features=tf.train.Features(feature=feature))

F_idx = []; list_of_files= []
for label in range(num_of_classes):
    # print('Class name and size: ' , CLASS_NAMES[label] , class_size[label])
    curr_class_idx = np.arange (class_size[label])
    # indeces_permuted = np.random.permutation(curr_class_idx)
    fold_size = int(np.round(  class_size[label] / nfolds ))
    F_idx.append(   curr_class_idx   )
    # print(  F_idx[label]   )
    path_name = datatsets_path+dataset + '/' + CLASS_NAMES[label] +'/'
    print('current class: ', label, ' name: ' ,  CLASS_NAMES[label], 'Size: ' , class_size[label], 'take in fold: ' , fold_size )
    data_dir = pathlib.Path(path_name)
    flist = list(data_dir.glob('*'+image_format)) 
    list_of_files.append(  flist   )

print(datatsets_path+dataset)






current class:  0  name:  lake Size:  700 take in fold:  35
current class:  1  name:  palace Size:  700 take in fold:  35
current class:  2  name:  chaparral Size:  700 take in fold:  35
current class:  3  name:  dense_residential Size:  700 take in fold:  35
current class:  4  name:  parking_lot Size:  700 take in fold:  35
current class:  5  name:  tennis_court Size:  700 take in fold:  35
current class:  6  name:  island Size:  700 take in fold:  35
current class:  7  name:  desert Size:  700 take in fold:  35
current class:  8  name:  meadow Size:  700 take in fold:  35
current class:  9  name:  beach Size:  700 take in fold:  35
current class:  10  name:  thermal_power_station Size:  700 take in fold:  35
current class:  11  name:  cloud Size:  702 take in fold:  35
current class:  12  name:  terrace Size:  700 take in fold:  35
current class:  13  name:  sea_ice Size:  700 take in fold:  35
current class:  14  name:  snowberg Size:  700 take in fold:  35
current class:  15  name:

In [ ]:



# print('num of classes: ', num_of_classes)
# folds_split_text_file = open("NWPU_RESISC45_folds_split_text_file.txt","w") 
# for fn in range(nfolds):
#     fixed_number = '%02d' %fn; 
#     tfrecord_train_file = datatsets_path+dataset + '_tfrecords/' +  'tfrecords.train.fold'+fixed_number
#     print(tfrecord_train_file)
#     folds_split_text_file.write('fold number: ' + fixed_number + ' \n')
#     for label in range(num_of_classes):
#       label_fixed_size_str = '%02d' %label; 
#       curr_class_idx = np.arange (class_size[label])
#       fold_size = int(np.round(  class_size[label] / nfolds ))
#       idx =   F_idx[label] ; # list of images idx permuted        
#       list_of_indeces = idx[ fn*fold_size:(fn+1)*fold_size  ]  # get a list of idx from this class for this fold  
#       if fn==(nfolds-1):
#         list_of_indeces = idx[ -fold_size:  ]; 
#       path_name = datatsets_path+dataset + '/' + CLASS_NAMES[label] +'/'
#       # path_name = datatsets_path+dataset + '/' + CLASS_NAMES[cn]
#       str_to_be_written_in_text_file = label_fixed_size_str
#       for i in range(fold_size):
#         file_num = '%04d'%list_of_indeces[i]
#         file_name = 'img'+file_num
#         file_name_with_path = path_name + file_name  +  image_format
#         str_to_be_written_in_text_file = str_to_be_written_in_text_file + ' ' + file_num

#       folds_split_text_file.write(str_to_be_written_in_text_file +'\n' )
#       # print('current class: '+label_fixed_size_str+ ' name: ' ,  CLASS_NAMES[label], 'Size: ' , class_size[label], 'take in fold: ' , fn*fold_size , 'to ', (fn+1)*fold_size)
#       print('current class: '+label_fixed_size_str+ ' name: ' ,  CLASS_NAMES[label], 'Size: ' , class_size[label], 'take in fold: ' + str_to_be_written_in_text_file)

# folds_split_text_file.close()

In [ ]:
# this code only shows the files to be written to disk, it does not actually write
path_to_fold_splits_files = '/content/gdrive/My Drive/RS_datasets/splits_of_datasets_into_folds/'  

print('num of classes: ', num_of_classes)
folds_split_text_file = open(path_to_fold_splits_files+folds_split_text_file_name,"r+") 
for fn in range(nfolds):
  next_line = folds_split_text_file.readline()
  fold_number = next_line[-4:-1]
  tfrecord_train_file = datatsets_path+dataset + '_tfrecords/' +  'tfrecords.train.fold'+fold_number
  print('foldNum: '+ fold_number+ tfrecord_train_file)
  for label in range(num_of_classes):
    className = CLASS_NAMES[label]
    next_line = folds_split_text_file.readline()
    image_numbers = next_line.split()
    print('foldNum: '+ fold_number+ 'classNum: ' + image_numbers[0]+ ' numImages: ',  len(image_numbers[1:])  , ' name: ' + CLASS_NAMES[label] ,  image_numbers[1:])


    path_name = datatsets_path+dataset + '/' + CLASS_NAMES[label] +'/'
    # path_name = datatsets_path+dataset + '/' + CLASS_NAMES[cn]
    for i in range(len(image_numbers)-1):
      file_name = 'img'+image_numbers[i+1]
      file_name_with_path = path_name + file_name  +  image_format
      # print( file_name_with_path )

folds_split_text_file.close()


In [ ]:

path_to_fold_splits_files = '/content/gdrive/My Drive/RS_datasets/splits_of_datasets_into_folds/'  

print('num of classes: ', num_of_classes)
folds_split_text_file = open(path_to_fold_splits_files+folds_split_text_file_name,"r+") 
for fn in range(nfolds):
  next_line = folds_split_text_file.readline()
  fold_number = next_line[-4:-1]
  if not(fn>=15 and fn<20):
    for label in range(num_of_classes):
        className = CLASS_NAMES[label]
        next_line = folds_split_text_file.readline()
        image_numbers = next_line.split()
        print('foldNum: '+ fold_number+ 'classNum: ' + image_numbers[0]+ ' numImages: ',  len(image_numbers[1:]) )
  else:
    tfrecord_train_file = datatsets_path+dataset + '_tfrecords/' +  'tfrecords.train.fold'+fold_number
    print('foldNum: '+ fold_number+ tfrecord_train_file)
    with tf.io.TFRecordWriter(tfrecord_train_file) as writer:
      for label in range(num_of_classes):
        className = CLASS_NAMES[label]
        next_line = folds_split_text_file.readline()
        image_numbers = next_line.split()
        print('foldNum: '+ fold_number+ 'classNum: ' + image_numbers[0]+ ' numImages: ',  len(image_numbers[1:])  , ' name: ' + CLASS_NAMES[label] , image_numbers[1:])


        path_name = datatsets_path+dataset + '/' + CLASS_NAMES[label] +'/'
        # path_name = datatsets_path+dataset + '/' + CLASS_NAMES[cn]
        for i in range(len(image_numbers)-1):
          file_name = 'img'+image_numbers[i+1]
          file_name_with_path = path_name + file_name  +  image_format
          # print( file_name_with_path )
          image_string = open(file_name_with_path, 'rb').read()
          tf_example = image_example(image_string, label)
          writer.write(tf_example.SerializeToString())

folds_split_text_file.close()



In [ ]:
# print('num of classes: ', num_of_classes)
# folds_split_text_file = open("folds_split_text_file.txt","w") 
# for fn in range(nfolds):
#     fixed_number = '%02d' %fn; folds_split_text_file.write('fold number: ' + fixed_number)
#     tfrecord_train_file = datatsets_path+dataset + '_tfrecords/' +  'tfrecords.train.fold'+fixed_number
#     print(tfrecord_train_file)
#     with tf.io.TFRecordWriter(tfrecord_train_file) as writer:
#       for label in range(num_of_classes):
#         label_fixed_size_str = '%02d' %label; 
#         curr_class_idx = np.arange (class_size[label])
#         fold_size = int(np.round(  class_size[label] / nfolds ))
#         idx =   F_idx[label] ; # list of images idx permuted        
#         list_of_indeces = idx[ fn*fold_size:(fn+1)*fold_size  ]  # get a list of idx from this class for this fold  
#         if fn==(nfolds-1):
#           list_of_indeces = idx[ -fold_size:  ]; 
#         path_name = datatsets_path+dataset + '/' + CLASS_NAMES[label] +'/'
#         # path_name = datatsets_path+dataset + '/' + CLASS_NAMES[cn]
#         str_to_be_written_in_text_file = label_fixed_size_str
#         for i in range(fold_size):
#           file_num = '%04d'%list_of_indeces[i]
#           file_name = 'img'+file_num
#           file_name_with_path = path_name + file_name  +  image_format
#           # print( 'tfrecords.train.fold'+str(fn), 'image: ', file_name_with_path)
#           image_string = open(file_name_with_path, 'rb').read()
#           tf_example = image_example(image_string, label)
#           writer.write(tf_example.SerializeToString())
#           str_to_be_written_in_text_file = str_to_be_written_in_text_file + ' ' + file_num

#         folds_split_text_file.write(str_to_be_written_in_text_file +'\n' )
#         # print('current class: '+label_fixed_size_str+ ' name: ' ,  CLASS_NAMES[label], 'Size: ' , class_size[label], 'take in fold: ' , fn*fold_size , 'to ', (fn+1)*fold_size)
#         print('current class: '+label_fixed_size_str+ ' name: ' ,  CLASS_NAMES[label], 'Size: ' , class_size[label], 'take in fold: ' + str_to_be_written_in_text_file)

# folds_split_text_file.close()

In [ ]:

#################################################################################################
####### This code can read image files from directories and save them as tfrecords
####### All images in a given class are saved in one tfrecords
#################################################################################################
def _bytes_feature(value):
  """Returns a bytes_list from a string / byte."""
  if isinstance(value, type(tf.constant(0))):
    value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
  """Returns a float_list from a float / double."""
  return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _int64_feature(value):
  """Returns an int64_list from a bool / enum / int / uint."""
  return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))


# Create a dictionary with features that may be relevant.
def image_example(image_string, label):
  image_shape = tf.image.decode_jpeg(image_string).shape

  feature = {
      #'height': _int64_feature(image_shape[0]),
      #'width': _int64_feature(image_shape[1]),
      #'depth': _int64_feature(image_shape[2]),
      'label': _int64_feature(label),
      'image_raw': _bytes_feature(image_string),
  }

  return tf.train.Example(features=tf.train.Features(feature=feature))


# Write the raw image files to `images.tfrecords`.
# First, process the two images into `tf.Example` messages.
# Then, write to a `.tfrecords` file.


for label_seq in range(num_of_classes):
  class_name = CLASS_NAMES0[label_seq]
  print(str(data_dir)+'/'+class_name)
  curr_class_dir = str(data_dir)+'/'+class_name
  list_of_images = list(glob.glob(curr_class_dir+'/*.png'))
  count_of_images = len(list_of_images)
  train_split = int(count_of_images/2)
  print(list_of_images)
  #train_split = 2
  # tfrecord_train_file = '/content/gdrive/My Drive/RS_datasets/NWPU_tfrecords/tfrecords.train.'+class_name
  # label = label_seq
  # with tf.io.TFRecordWriter(tfrecord_train_file) as writer:
  #   for i in range(train_split):
  #     filename = list_of_images[i]; 
  #     image_string = open(filename, 'rb').read()
  #     tf_example = image_example(image_string, label)
  #     writer.write(tf_example.SerializeToString())
  # tfrecord_validation_file = '/content/gdrive/My Drive/RS_datasets/NWPU_tfrecords/tfrecords.test.'+class_name
  # with tf.io.TFRecordWriter(tfrecord_validation_file) as writer:
  #   for i in range(train_split):
  #     filename = list_of_images[i+train_split]; 
  #     image_string = open(filename, 'rb').read()
  #     tf_example = image_example(image_string, label)
  #     writer.write(tf_example.SerializeToString())
